In [1]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from collections import defaultdict

model = YOLO('yolov8n.pt')
car_class_id = 2  # Class ID for cars
bus_class_id = 5  # Class ID for buses
truck_class_id = 7  # Class ID for trucks
confidence_threshold = 0.5

In [4]:
cap=cv2.VideoCapture('highway.mp4')

Area_of_Interest = [(200,210), (410,210),(360,330),(1,320)] # top-left -> top-right -> bottom-right -> bottom-left

#add speed
fps = cap.get(cv2.CAP_PROP_FPS)  # Get the frame rate of the video
distance_per_pixel = 0.05

previous_positions = defaultdict(list)


while True:
    cap_check, frame = cap.read()
    if not cap_check:
        break
    
    frame = cv2.resize(frame, (0,0), fx=0.5, fy=0.5)
    cv2.imwrite('saved_image.jpg', frame)
    
    height, width = frame.shape[:2]
    print(f'Resized Frame Size: Width = {width}, Height = {height}')

    results = model(frame)
    detections = results[0].boxes

    current_positions = defaultdict(tuple)

    for box in detections.data:

        x1, y1, x2, y2, conf, cls = box.tolist()
        cx = int(x1+x2) //2
        cy = int(y1+y2) //2

        if conf >= confidence_threshold and (cls == car_class_id or cls == truck_class_id or cls == bus_class_id):
                
            # return 1 if inside, 0 if on the edge, -1 if outside 
            result = cv2.pointPolygonTest(np.array(Area_of_Interest,np.int32),(int(cx),int(cy)),False) 

            if(result >= 0):
                # add speed
                object_id = int(cls)
                current_positions[object_id] = (cx, cy)
    
                # Draw the bounding box
                cv2.circle(frame, (cx, cy), 5, (0, 255, 0), -1)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                # Put class ID or name on the frame
                cv2.putText(frame, f'Conf: {conf:.2f}', (int(x1), int(y1) - 10), #ID: {int(cls)} 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                
                #add speed
                if previous_positions[object_id]:
                    # Get previous position
                    prev_x, prev_y = previous_positions[object_id]

                    # Calculate the distance moved in pixels
                    pixel_distance = np.sqrt((cx - prev_x) ** 2 + (cy - prev_y) ** 2)

                    # Convert pixel distance to real-world distance (in meters)
                    real_world_distance = pixel_distance * distance_per_pixel

                    # Calculate speed (distance/time), time = 1/fps
                    speed = real_world_distance * fps  # Speed in meters per second

                    # Display speed on the frame (convert to km/h by multiplying by 3.6)
                    cv2.putText(frame, f'Speed: {speed * 3.6:.2f} km/h', (int(x1), int(y2) + 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 200, 255), 2)
                
    previous_positions = current_positions

    cv2.polylines(frame,[np.array(Area_of_Interest,np.int32)],True,(0,255,255),2)

    # Display the frame
    cv2.imshow('frame',frame)

    key = cv2.waitKey(33) & 0xFF

    if key == 27:
        break
    elif key == ord('s') or key == ord('S'):  # Save frame when 'S' is pressed
        cv2.imwrite('captured_frame.jpg', frame)
        print('Frame captured and saved as captured_frame.jpg')

cap.release()
cv2.destroyAllWindows()


0: 384x640 8 cars, 1 truck, 50.5ms
Speed: 2.1ms preprocess, 50.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)



Resized Frame Size: Width = 960, Height = 540
Resized Frame Size: Width = 960, Height = 540


0: 384x640 8 cars, 1 truck, 61.6ms
Speed: 2.0ms preprocess, 61.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 truck, 64.7ms
Speed: 1.0ms preprocess, 64.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 52.5ms
Speed: 1.6ms preprocess, 52.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


Resized Frame Size: Width = 960, Height = 540
Resized Frame Size: Width = 960, Height = 540



0: 384x640 6 cars, 1 truck, 56.5ms
Speed: 1.5ms preprocess, 56.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 truck, 53.8ms
Speed: 2.0ms preprocess, 53.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


...

Resized Frame Size: Width = 960, Height = 540
Resized Frame Size: Width = 960, Height = 540


0: 384x640 11 cars, 1 bus, 58.3ms
Speed: 1.7ms preprocess, 58.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 bus, 56.9ms
Speed: 1.0ms preprocess, 56.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 bus, 57.8ms


Resized Frame Size: Width = 960, Height = 540
Resized Frame Size: Width = 960, Height = 540


Speed: 2.0ms preprocess, 57.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 bus, 55.7ms
Speed: 1.0ms preprocess, 55.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


Resized Frame Size: Width = 960, Height = 540
